In [1]:
from causaldag import unknown_target_igsp
import causaldag as cd
import random
from causaldag.utils.ci_tests import gauss_ci_suffstat, gauss_ci_test, MemoizedCI_Tester
from causaldag.utils.invariance_tests import gauss_invariance_suffstat, gauss_invariance_test, MemoizedInvarianceTester
import pandas
import numpy as np
import time

data = pandas.read_csv("./data/sachs_data_sel.csv")
Ts = pandas.read_csv("./data/sachs_Ts.csv")["x"]

obs_samples = np.array(data.iloc[np.where(Ts == 0)])

seed_number = 42

n = 11

diffTs = np.array(list(set(Ts)))
diffTs = diffTs[np.where(diffTs > 0)]

iv_samples_list = [np.array(data.iloc[np.where(Ts == ii)]) for ii in diffTs]


# Form sufficient statistics
obs_suffstat = gauss_ci_suffstat(obs_samples)
invariance_suffstat = gauss_invariance_suffstat(obs_samples, iv_samples_list)
setting_list = [dict(known_interventions=[]) for ii in diffTs]


SP_alphas = np.array([0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4])
DAGstring = [""] * len(SP_alphas)

print(seed_number)

start_time = time.time()
# Run UT-IGSP
# Create conditional independence tester and invariance tester
for ii in range(len(SP_alphas)):
    alpha = SP_alphas[ii]
    print(alpha)
    alpha_inv = 1e-20 #alpha
    ci_tester = MemoizedCI_Tester(gauss_ci_test, obs_suffstat, alpha=alpha)
    invariance_tester = MemoizedInvarianceTester(gauss_invariance_test, invariance_suffstat, alpha=alpha_inv)
    try:
        est_dag, est_targets_list = unknown_target_igsp(setting_list, set(range(n)), ci_tester, invariance_tester)
    except:
        est_dag = "error"
        print("SVD error")
    DAGstring[ii] = est_dag

time_taken = time.time() - start_time
print(time_taken)

DAGs = pandas.DataFrame(DAGstring)

DAGs.to_csv("./sachsresults/sachs_UT-IGSP_DAGs.csv", header=False, index=False)



42
0.0001
0.0005
0.001
0.005
0.01
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
1.902881145477295
